In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##데이터준비

###데이터 불러오기

In [2]:
import numpy as np
import pandas as pd

In [3]:
# 상품분류통합된 구매정보 데이터 불러오기
npurch = pd.read_csv('/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/df_purchase_trl.csv',
                    encoding='cp949', chunksize = 1000000) #100만개 단위로 끊기, 실제 transaction 데이터
chunk = list(npurch)

print(len(chunk)) # 몇개 chunk인지 출력

npurch = pd.concat(chunk)  # 데이터 넣기
npurch['구매일자'] = pd.to_datetime(npurch['구매일자'],format='%Y-%m-%d')
npurch.shape

29


(28593030, 13)

##피처0. 고객 분기별 중분류상품 총구매금액 등급 -> 증감지수

###사전작업

In [69]:
# 중분류명 담은 리스트 생성
li = npurch['중분류명'].unique()
print(li)

['N음료류' '주류' '일용잡화' '주방용품' 'N스낵류' '가공식품' '생활잡화' '농산물' '스포츠' '육류' '차/커피'
 '화장품' 'N냉동류' '섬유잡화' '젓갈/반찬' '식기' '기타' '액세서리' '란제리/내의' 'N냉장류' '건강식품' '수산품'
 '캐주얼' '구두' '축산가공' 'N푸드코트' '전자/전기' '문구/팬시' '기타의류' '남성 트랜디' '아동' 'N자동차용품'
 '침구/수예' 'N도서/음반' '가구' '소형가전' '준보석/시계' '명품' '남성캐주얼' '골프용품' '피혁잡화' '남성정장'
 '셔츠' '통신/컴퓨터' '트래디셔널' '보석' '커리어' '주방가전' '시티웨어' '디자이너' '모피/피혁' '대형가전' '교복'
 '악기' '블라우스' '웨딩']


In [104]:
# 고객번호 drop 없이 출력하기 위해 틀 생성
import numpy as np
basedf = pd.DataFrame({'고객번호':np.arange(1, 19327+1)}, index=np.arange(19327))
print(basedf.shape)
basedf.head()

(19327, 1)


,고객번호
0,1
1,2
2,3
3,4
4,5


###중분류별 증감지수

In [100]:
# 테스트

i = 0
mom_df = npurch[npurch['중분류명'] == li[i]]
mom_df.drop(['제휴사', '구매시간', '점포', '거주지역', '거주지', '점포지역', '월', '년도', '4분기'], axis=1, inplace=True)

df_final = pd.merge(mom_df, quan_grade(mom_df, '2014-01-01', '2014-03-31', 'grade14_1'), on='고객번호', how='right')
df_final = pd.merge(df_final, quan_grade(mom_df, '2014-04-01', '2014-06-30', 'grade14_2'), on='고객번호', how='right')
df_final = pd.merge(df_final, quan_grade(mom_df, '2014-07-01', '2014-09-30', 'grade14_3'), on='고객번호', how='right')
df_final = pd.merge(df_final, quan_grade(mom_df, '2014-10-01', '2014-12-31', 'grade14_4'), on='고객번호', how='right')
df_final = pd.merge(df_final, quan_grade(mom_df, '2015-01-01', '2015-03-31', 'grade15_1'), on='고객번호', how='right')
df_final = pd.merge(df_final, quan_grade(mom_df, '2015-04-01', '2015-06-30', 'grade15_2'), on='고객번호', how='right')

df_final.iloc[:, 4:] = df_final.iloc[:, 4:].fillna(0)

df_final['fluctuation'] = (df_final['grade14_1'] - df_final['grade14_2']) + \
                          (df_final['grade14_2'] - df_final['grade14_3']) + \
                          (df_final['grade14_3'] - df_final['grade14_4']) + \
                          (df_final['grade14_4'] - df_final['grade15_1']) + \
                          (df_final['grade15_1'] - df_final['grade15_2'])

grouped = df_final.groupby(['고객번호', 'fluctuation'], as_index=False)[['구매금액']].sum()

In [109]:
# 고객번호 drop 없이 merge
df_final = pd.merge(basedf, grouped, on='고객번호', how='left')[['고객번호', 'fluctuation']]
df_final.shape

(19327, 2)

In [111]:
# 파일로 저장
df_final.to_csv(f'/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/카쓰/fluctuation_{li[i]}.csv', index=False)

- 문제점
1. 고객의 구매이력이 없는 분기에는 0등급인데, 다음 분기에 영향을 크게 미쳐서 결국 fluctuation 값이 심각하게 떨어진다.
2. 해당 중분류상품을 구매한 이력이 없는 경우 fluctuation이 결측치이다.
3. 원본 데이터프레임에는 1번 고객의 구매금액이 있다. 그러나 groupby 후에는 구매금액이 0으로 나온다.

In [79]:
mom_df[mom_df['고객번호']==1]

,고객번호,구매일자,구매금액,중분류명
8219557,1,2015-04-25,6600,N음료류
8219587,1,2014-10-09,8400,N음료류
8220206,1,2014-08-13,11280,N음료류
8220385,1,2015-01-08,11280,N음료류


In [103]:
grouped.head()

,고객번호,fluctuation,구매금액
0,1,-3.0,0.0
1,3,2.0,251490.0
2,6,8.0,605290.0
3,9,-7.0,0.0
4,10,1.0,56140.0


### for문 반복작업

1부터 55까지 바꾸며 돌려서 파일 저장.

In [113]:
len(li)

56

In [122]:
# 중분류명에 '/'로 Error 피하기 위한 작업
x, y = li[10].split('/')
print(x+y)

차커피


In [132]:
li

array(['N음료류', '주류', '일용잡화', '주방용품', 'N스낵류', '가공식품', '생활잡화', '농산물', '스포츠',
       '육류', '차/커피', '화장품', 'N냉동류', '섬유잡화', '젓갈/반찬', '식기', '기타', '액세서리',
       '란제리/내의', 'N냉장류', '건강식품', '수산품', '캐주얼', '구두', '축산가공', 'N푸드코트',
       '전자/전기', '문구/팬시', '기타의류', '남성 트랜디', '아동', 'N자동차용품', '침구/수예',
       'N도서/음반', '가구', '소형가전', '준보석/시계', '명품', '남성캐주얼', '골프용품', '피혁잡화',
       '남성정장', '셔츠', '통신/컴퓨터', '트래디셔널', '보석', '커리어', '주방가전', '시티웨어',
       '디자이너', '모피/피혁', '대형가전', '교복', '악기', '블라우스', '웨딩'], dtype=object)

In [131]:
for i in range(52, len(li)):
  mom_df = npurch[npurch['중분류명'] == li[i]]
  mom_df.drop(['제휴사', '구매시간', '점포', '거주지역', '거주지', '점포지역', '월', '년도', '4분기'], axis=1, inplace=True)

  df_final = pd.merge(mom_df, quan_grade(mom_df, '2014-01-01', '2014-03-31', 'grade14_1'), on='고객번호', how='right')
  df_final = pd.merge(df_final, quan_grade(mom_df, '2014-04-01', '2014-06-30', 'grade14_2'), on='고객번호', how='right')
  df_final = pd.merge(df_final, quan_grade(mom_df, '2014-07-01', '2014-09-30', 'grade14_3'), on='고객번호', how='right')
  df_final = pd.merge(df_final, quan_grade(mom_df, '2014-10-01', '2014-12-31', 'grade14_4'), on='고객번호', how='right')
  df_final = pd.merge(df_final, quan_grade(mom_df, '2015-01-01', '2015-03-31', 'grade15_1'), on='고객번호', how='right')
  df_final = pd.merge(df_final, quan_grade(mom_df, '2015-04-01', '2015-06-30', 'grade15_2'), on='고객번호', how='right')

  df_final.iloc[:, 4:] = df_final.iloc[:, 4:].fillna(0)

  df_final['fluctuation'] = (df_final['grade14_1'] - df_final['grade14_2']) + \
                            (df_final['grade14_2'] - df_final['grade14_3']) + \
                            (df_final['grade14_3'] - df_final['grade14_4']) + \
                            (df_final['grade14_4'] - df_final['grade15_1']) + \
                            (df_final['grade15_1'] - df_final['grade15_2'])

  grouped = df_final.groupby(['고객번호', 'fluctuation'], as_index=False)[['구매금액']].sum()

  df_final = pd.merge(basedf, grouped, on='고객번호', how='left')[['고객번호', 'fluctuation']]
  df_final.shape

  if len(li[i].split('/')) > 1:
    x, y = li[10].split('/')
    df_final.to_csv(f'/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/카쓰/fluctuation_{x+y}.csv', index=False)
  else:
    df_final.to_csv(f'/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/카쓰/fluctuation_{li[i]}.csv', index=False)

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


ValueError: ignored

In [128]:
i # for문 멈춘 지점
# ValueError: Bin edges must be unique: array([ 0., nan, nan, nan, nan, nan, nan, nan, nan, nan]).
# You can drop duplicate edges by setting the 'duplicates' kwarg

# ['교복', '악기', '블라우스', '웨딩'] 작업 불가. 값이 없나? 아하, 가격대를 나누었는데 그만큼의 값이 안 나오는 건가?

52

In [135]:
npurch[npurch['중분류명'] == '교복']['구매금액'].unique() # 아닌데..

array([ 16000, 409000, 183000, 105000, 353000, 250000, 103000, 278000,
        63000, 347500, 168000, 160000,  85000, 475000,  58000, 249000,
       144000, 320000, 350000, 128000, 165000,  66000,  56000, 180000,
       420000, 190000,  25000, 430000, 217000,  44000, 175000,  23000,
       352000,  94000,  27000, 134000, 248000,  65000, 399000, 413000,
       223000, 421000, 194000, 302500, 140000, 193000,  50000, 322000,
       245000, 302000,  32000, 355500,  35000, 229000, 410800, 479000,
         5000, 481000,  60000,  42000, 158000,  30000, 106000, 383000,
       354000, 192000, 381000, 224000, 408000, 357000,  96000, 149000,
       349000, 170000, 283000,  53000, 120000, 280000, 274000, 255000,
       281000,   6000, 102000,  40000, 382000, 372400, 429000,  48000,
       253000,  88000,  54000, 179000,  33000, 188000, 319000, 148000,
       130000,  24000, 221000, 367000,  90000, 107000, 100000, 239000,
       306000, 206000, 438000, 358000,  80000, 121000,  55000,  20000,
      

In [141]:
import glob

file_name = glob.glob('/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/카쓰/*.csv')
print(len(file_name))
file_name

45


['/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/카쓰/매출대비구매비율.csv',
 '/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/카쓰/fluctuation_N음료류.csv',
 '/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/카쓰/fluctuation_주류.csv',
 '/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/카쓰/fluctuation_일용잡화.csv',
 '/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/카쓰/fluctuation_주방용품.csv',
 '/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/카쓰/fluctuation_N스낵류.csv',
 '/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/카쓰/fluctuation_가공식품.csv',
 '/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/카쓰/fluctuation_생활잡화.csv',
 '/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/카쓰/fluctuation_농산물.csv',
 '/content/d

###코딩과정

In [22]:
mom_df = npurch.copy()

In [24]:
df = mom_df[mom_df['중분류명'] == li[0]]
df.head()

,제휴사,고객번호,구매일자,구매시간,구매금액,중분류명,점포,거주지역,거주지,점포지역,월,년도,4분기
0,B,17218,2014-02-22,20,2420,N음료류,B44,100.0,경기,경기,2,2014,1
7,B,17218,2015-06-19,21,2500,N음료류,B44,100.0,경기,경기,6,2015,2
16,B,17218,2015-05-30,16,910,N음료류,B44,100.0,경기,경기,5,2015,2
22,B,17218,2014-06-11,18,760,N음료류,B44,100.0,경기,경기,6,2014,2
33,B,17218,2014-03-21,18,380,N음료류,B44,100.0,경기,경기,3,2014,1


In [25]:
df = df[df['구매일자'].between('2014-01-01', '2014-03-31')]
df.shape

(103362, 13)

In [26]:
df = df.groupby(['고객번호'], as_index=False)[['구매금액']].sum()
df.head()

,고객번호,구매금액
0,3,23100
1,6,115400
2,9,2440
3,10,8420
4,11,18520


In [28]:
p_li = [0.04, 0.11, 0.23, 0.40, 0.60, 0.77, 0.89, 0.96] # 퍼센트
bins = [0] + [df['구매금액'].quantile(q=p_li[i]) for i in range(8)] + [df['구매금액'].quantile(1)] # 등급별 기준값
labels = [1, 2, 3, 4, 5, 6, 7, 8, 9] # 등급
df['grade14_1'] = pd.cut(df['구매금액'], bins=bins, labels=labels)
df['grade14_1'] = pd.to_numeric(df['grade14_1'])

In [29]:
df.head()

,고객번호,구매금액,grade14_1
0,3,23100,6
1,6,115400,9
2,9,2440,2
3,10,8420,4
4,11,18520,5


In [30]:
df.drop('구매금액', axis=1, inplace=True)
df.head()

,고객번호,grade14_1
0,3,6
1,6,9
2,9,2
3,10,4
4,11,5


##피처1. 고객의 중분류상품 구매금액/ 중분류상품 총판매금액


In [6]:
train_purch = npurch[npurch['구매일자'].between('2014-01-01', '2015-06-30')]
train_purch.head()

,제휴사,고객번호,구매일자,구매시간,구매금액,중분류명,점포,거주지역,거주지,점포지역,월,년도,4분기
0,B,17218,2014-02-22,20,2420,N음료류,B44,100.0,경기,경기,2,2014,1
1,B,17218,2014-02-22,20,1070,주류,B44,100.0,경기,경기,2,2014,1
2,B,17218,2014-02-22,20,8060,주류,B44,100.0,경기,경기,2,2014,1
3,B,17218,2014-02-22,20,6000,일용잡화,B44,100.0,경기,경기,2,2014,1
5,B,17218,2014-12-09,19,3440,N스낵류,B44,100.0,경기,경기,12,2014,4


In [7]:
# 각 고객의 중분류별 총구매금액
grouped1 = train_purch.groupby(['고객번호', '중분류명'], as_index=False)[['구매금액']].sum()
grouped1.head()

,고객번호,중분류명,구매금액
0,1,N냉동류,13960
1,1,N냉장류,43120
2,1,N스낵류,32300
3,1,N음료류,37560
4,1,N푸드코트,8000


In [ ]:
# 중분류별 총매출
grouped2 = train_purch.groupby(['중분류명'])[['구매금액']].sum()
grouped2.head()

,구매금액
중분류명,
N냉동류,2502559104
N냉장류,1932757205
N도서/음반,1497169045
N스낵류,6185018005
N음료류,3033844720


In [ ]:
# merge 로 합치기
df = pd.merge(grouped1, grouped2, on='중분류명', how='left')

# 총매출 대비 고객의 총구매액
df['매출대비구매비율'] = df['구매금액_x'] / df['구매금액_y']

In [ ]:
df.head()

,고객번호,중분류명,구매금액_x,구매금액_y,매출대비구매비율
0,1,N냉동류,13960,2502559104,0.000006
1,1,N냉장류,43120,1932757205,0.000022
2,1,N스낵류,32300,6185018005,0.000005
3,1,N음료류,37560,3033844720,0.000012
4,1,N푸드코트,8000,962241397,0.000008


In [ ]:
df = df.pivot(index = '고객번호', columns = '중분류명', values='매출대비구매비율').fillna(0)
df.head()

중분류명,N냉동류,N냉장류,N도서/음반,N스낵류,N음료류,N자동차용품,N푸드코트,가공식품,가구,건강식품,...,준보석/시계,차/커피,축산가공,침구/수예,캐주얼,커리어,통신/컴퓨터,트래디셔널,피혁잡화,화장품
고객번호,,,,,,,,,,,,,,,,,,,,,
1,0.000006,0.000022,0.0,0.000005,0.000012,0.0,0.000008,0.000066,0.0,0.000452,...,0.000259,0.000056,0.000166,0.000924,0.000196,0.000257,0.000026,0.000248,0.000096,1.057061e-04
2,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000044,0.0,0.000033,...,0.000000,0.000097,0.000374,0.000108,0.000006,0.000033,0.000000,0.000092,0.000040,5.897167e-04
3,0.000002,0.000043,0.0,0.000007,0.000052,0.0,0.000000,0.000012,0.0,0.000000,...,0.000000,0.000019,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.723393e-07
4,0.000055,0.000023,0.0,0.000001,0.000003,0.0,0.000000,0.000028,0.0,0.000011,...,0.000000,0.000032,0.000107,0.000071,0.000015,0.000067,0.000000,0.000000,0.000028,2.533276e-05
5,0.000000,0.000000,0.0,0.000002,0.000000,0.0,0.000000,0.000029,0.0,0.000253,...,0.000000,0.000044,0.001104,0.000036,0.000002,0.000000,0.000000,0.000013,0.000000,4.973731e-05


In [ ]:
df.columns

Index(['N냉동류', 'N냉장류', 'N도서/음반', 'N스낵류', 'N음료류', 'N자동차용품', 'N푸드코트', '가공식품',
       '가구', '건강식품', '골프용품', '교복', '구두', '기타', '기타의류', '남성 트랜디', '남성정장',
       '남성캐주얼', '농산물', '대형가전', '디자이너', '란제리/내의', '명품', '모피/피혁', '문구/팬시', '보석',
       '생활잡화', '섬유잡화', '셔츠', '소형가전', '수산품', '스포츠', '시티웨어', '식기', '아동', '악기',
       '액세서리', '웨딩', '육류', '일용잡화', '전자/전기', '젓갈/반찬', '주류', '주방가전', '주방용품',
       '준보석/시계', '차/커피', '축산가공', '침구/수예', '캐주얼', '커리어', '통신/컴퓨터', '트래디셔널',
       '피혁잡화', '화장품'],
      dtype='object', name='중분류명')

In [ ]:
len(df.columns)

55

In [ ]:
# df.to_csv('/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/카쓰/매출대비구매비율.csv')

##피처2. 고객 분기별 총구매금액 등급 -> 증감지수

In [ ]:
npurch.drop(['제휴사', '구매시간', '중분류명', '점포', '거주지역', '거주지', '점포지역', '월', '년도', '4분기'], axis=1, inplace=True)

In [10]:
def quan_grade(mom_df, quan_start, quan_end, col_name):

  df = mom_df[mom_df['구매일자'].between(quan_start, quan_end)]

  # gorupby
  df = df.groupby(['고객번호'], as_index=False)[['구매금액']].sum()

  # pd.cut
  p_li = [0.04, 0.11, 0.23, 0.40, 0.60, 0.77, 0.89, 0.96] # 퍼센트
  bins = [0] + [df['구매금액'].quantile(q=p_li[i]) for i in range(8)] + [df['구매금액'].quantile(1)] # 등급별 기준값
  labels = [1, 2, 3, 4, 5, 6, 7, 8, 9] # 등급

  df[col_name] = pd.cut(df['구매금액'], bins=bins, labels=labels)
  df[col_name] = pd.to_numeric(df[col_name])

  # unvaluable column out
  df.drop('구매금액', axis=1, inplace=True)

  return df

In [ ]:
df_final = pd.merge(npurch, quan_grade(npurch, '2014-01-01', '2014-03-31', 'grade14_1'), on='고객번호', how='right')
df_final = pd.merge(df_final, quan_grade(npurch, '2014-04-01', '2014-06-30', 'grade14_2'), on='고객번호', how='right')
df_final = pd.merge(df_final, quan_grade(npurch, '2014-07-01', '2014-09-30', 'grade14_3'), on='고객번호', how='right')
df_final = pd.merge(df_final, quan_grade(npurch, '2014-10-01', '2014-12-31', 'grade14_4'), on='고객번호', how='right')
df_final = pd.merge(df_final, quan_grade(npurch, '2015-01-01', '2015-03-31', 'grade15_1'), on='고객번호', how='right')
df_final = pd.merge(df_final, quan_grade(npurch, '2015-04-01', '2015-06-30', 'grade15_2'), on='고객번호', how='right')
# df_final = pd.merge(df_final, quan_grade(npurch, '2015-07-01', '2015-09-30', 'grade15_3'), on='고객번호', how='right')
# df_final = pd.merge(df_final, quan_grade(npurch, '2015-10-01', '2015-12-31', 'grade15_4'), on='고객번호', how='right')

In [ ]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28269669 entries, 0 to 28269668
Data columns (total 9 columns):
 #   Column     Dtype         
---  ------     -----         
 0   고객번호       int64         
 1   구매일자       datetime64[ns]
 2   구매금액       float64       
 3   grade14_1  float64       
 4   grade14_2  float64       
 5   grade14_3  float64       
 6   grade14_4  float64       
 7   grade15_1  float64       
 8   grade15_2  int64         
dtypes: datetime64[ns](1), float64(6), int64(2)
memory usage: 2.1 GB


In [ ]:
df_final.iloc[:, 3:] = df_final.iloc[:, 3:].fillna(0)
# df_final.iloc[:, 3:]

In [ ]:
df_final['fluctuation'] = (df_final['grade14_1'] - df_final['grade14_2']) + \
                          (df_final['grade14_2'] - df_final['grade14_3']) + \
                          (df_final['grade14_3'] - df_final['grade14_4']) + \
                          (df_final['grade14_4'] - df_final['grade15_1']) + \
                          (df_final['grade15_1'] - df_final['grade15_2'])

In [ ]:
df_final

,고객번호,구매일자,구매금액,grade14_1,grade14_2,grade14_3,grade14_4,grade15_1,grade15_2,fluctuation
0,1,2014-01-18,10000.0,8.0,8.0,8.0,9.0,8.0,7,1.0
1,1,2014-02-03,129000.0,8.0,8.0,8.0,9.0,8.0,7,1.0
2,1,2014-04-25,7000.0,8.0,8.0,8.0,9.0,8.0,7,1.0
3,1,2014-04-25,2650.0,8.0,8.0,8.0,9.0,8.0,7,1.0
4,1,2014-04-25,9800.0,8.0,8.0,8.0,9.0,8.0,7,1.0
...,...,...,...,...,...,...,...,...,...,...
28269664,19379,NaT,NaN,0.0,0.0,1.0,3.0,3.0,4,-4.0
28269665,19380,NaT,NaN,0.0,0.0,0.0,1.0,1.0,1,-1.0
28269666,19381,NaT,NaN,0.0,0.0,0.0,0.0,3.0,4,-4.0
28269667,19382,NaT,NaN,0.0,0.0,0.0,0.0,2.0,3,-3.0


In [ ]:
grouped = df_final.groupby(['고객번호', 'fluctuation'], as_index=False)[['구매금액']].sum()
# grouped.to_csv('/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/카쓰/train_data_fluctuation.csv',
#                index=False)


In [ ]:
grouped

,고객번호,fluctuation,구매금액
0,1,1.0,85042550.0
1,2,0.0,99880760.0
2,3,0.0,3165167.0
3,4,-1.0,16693810.0
4,5,4.0,13237070.0
...,...,...,...
19301,19379,-4.0,0.0
19302,19380,-1.0,0.0
19303,19381,-4.0,0.0
19304,19382,-3.0,0.0


In [ ]:
# 램부족 위험대비 파일 저장
# df_final.to_csv('/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/5팀_공유폴더/카쓰/train_data_quan_grade.csv',
#                 index=False)
